# SQL translate

Translate natural language to SQL queries.

## Settings

| Setting | Value |
| --- | --- |
| Model | text-davinci-003 |
| Max tokens | 150 |
| Temperature | 0 | 
| Top p | 1 | 
| Frequency penalty | 0.0 |
| Presence penalty | 0.0 |
| Stop sequence | `#` `;` |

## Prompt

```text
### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months
SELECT
```

## Install Azure OpenAI Service .NET SDK

[Azure.AI.OpenAI NuGet package](https://www.nuget.org/packages/Azure.AI.OpenAI/)

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.5"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.5

In [2]:
using System;
using Azure.AI.OpenAI;

## Configure Azure OpenAI client

In [3]:
var AOAI_ENDPOINT = Environment.GetEnvironmentVariable("AOAI_ENDPOINT");
var AOAI_KEY = Environment.GetEnvironmentVariable("AOAI_KEY");
var AOAI_DEPLOYMENTID = Environment.GetEnvironmentVariable("AOAI_DEPLOYMENTID");

In [4]:
var endpoint = new Uri(AOAI_ENDPOINT);
var credentials = new Azure.AzureKeyCredential(AOAI_KEY);
var openAIClient = new OpenAIClient(endpoint, credentials);

## Define prompt

In [5]:
var prompt = 
    """
    ### Postgres SQL tables, with their properties:
    #
    # Employee(id, name, department_id)
    # Department(id, name, address)
    # Salary_Payments(id, employee_id, amount, date)
    #
    ### A query to list the names of the departments which employed more than 10 employees in the last 3 months
    SELECT
    """;

## Configure completion options

In [6]:
var completionOptions = new CompletionsOptions
{
    MaxTokens=150,
    Temperature=0f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
};

completionOptions.StopSequences.Add("#");
completionOptions.StopSequences.Add(";");
completionOptions.Prompts.Add(prompt);

## Generate completions

In [7]:
Completions response = await openAIClient.GetCompletionsAsync(AOAI_DEPLOYMENTID,completionOptions);

In [8]:
response.Choices[0].Text

 d.name 
FROM Department d 
INNER JOIN Employee e 
ON d.id = e.department_id 
INNER JOIN Salary_Payments sp 
ON e.id = sp.employee_id 
WHERE sp.date > NOW() - INTERVAL '3 months' 
GROUP BY d.name 
HAVING COUNT(*) > 10